In [118]:
import re
from tqdm import trange
from rouge_score import rouge_scorer
import numpy as np
import torch

### Examples

In [111]:
mclas_pred_file_path='./mclas/test_mclas.250000.canstr'
ncls_pred_file_path='./ncls/test_ncls.250000.canstr'
otkd_pred_file_path='./otkd/test.250000.canstr'

mclas_gold_file_path='./mclas/test_mclas.250000.goldstr'
ncls_gold_file_path='./ncls/test_ncls.250000.goldstr'
otkd_gold_file_path='./otkd/test.250000.goldstr'

mono_gold_file_path='./mclas/test_mclas.250000.gold_eng'

In [115]:
with open(mclas_gold_file_path, 'r') as f: mclas_gold_lines=f.readlines()
for i in trange(len(mclas_gold_lines)): mclas_gold_lines[i]=mclas_gold_lines[i].strip()
    
with open(ncls_gold_file_path, 'r') as f: ncls_gold_lines=f.readlines()
for i in trange(len(ncls_gold_lines)): ncls_gold_lines[i]=re.sub(r'\[unused[0-9]\]','',ncls_gold_lines[i]).replace('[PAD]','').strip()    
    
with open(otkd_gold_file_path, 'r') as f: otkd_gold_lines=f.readlines()
for i in trange(len(otkd_gold_lines)): otkd_gold_lines[i]=re.sub(r'\[unused[0-9]\]','',otkd_gold_lines[i]).replace('[PAD]','').strip()    

with open(mono_gold_file_path, 'r') as f: mono_gold_lines=f.readlines()
for i in trange(len(mono_gold_lines)): mono_gold_lines[i]=mono_gold_lines[i].strip()    
    
with open(mclas_pred_file_path, 'r') as f: mclas_lines=f.readlines()
for i in trange(len(mclas_lines)): mclas_lines[i]=mclas_lines[i].strip()    
    
with open(ncls_pred_file_path, 'r') as f: ncls_lines=f.readlines()
for i in trange(len(ncls_lines)): ncls_lines[i]=re.sub(r'\[unused[0-9]\]','',ncls_lines[i]).strip()    
    
with open(otkd_pred_file_path, 'r') as f: otkd_lines=f.readlines()
for i in trange(len(otkd_lines)): otkd_lines[i]=re.sub(r'\[unused[0-9]\]','',otkd_lines[i]).strip()
    
mclas_sorted_args=np.argsort(mclas_gold_lines)
ncls_sorted_args=np.argsort(ncls_gold_lines)
otkd_sorted_args=np.argsort(otkd_gold_lines)

pred_mclas_lines=[]
pred_ncls_lines=[]
pred_otkd_lines=[]
label_mclas_lines=[]
label_ncls_lines=[]
label_otkd_lines=[]
label_mono_lines=[]

for i in mclas_sorted_args: pred_mclas_lines.append(mclas_lines[i])
for i in ncls_sorted_args: pred_ncls_lines.append(ncls_lines[i])    
for i in otkd_sorted_args: pred_otkd_lines.append(otkd_lines[i])
for i in mclas_sorted_args: label_mclas_lines.append(mclas_gold_lines[i])
for i in ncls_sorted_args: label_ncls_lines.append(ncls_gold_lines[i])    
for i in otkd_sorted_args: label_otkd_lines.append(otkd_gold_lines[i])
for i in mclas_sorted_args: label_mono_lines.append(mono_gold_lines[i])

100%|██████████| 2997/2997 [00:00<00:00, 246864.28it/s]


In [113]:
mclas_rouge_scorer_rouge1_list = []
ncls_rouge_scorer_rouge1_list = []
otkd_rouge_scorer_rouge1_list = []

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'], use_stemmer=True)
for i in trange(len(mclas_gold_lines)): mclas_rouge_scorer_rouge1_list.append(scorer.score(pred_mclas_lines[i].strip(),label_mclas_lines[i].strip())['rouge1'].fmeasure)
for i in trange(len(ncls_gold_lines)): ncls_rouge_scorer_rouge1_list.append(scorer.score(pred_ncls_lines[i].strip(),label_ncls_lines[i].strip())['rouge1'].fmeasure)
for i in trange(len(otkd_gold_lines)): otkd_rouge_scorer_rouge1_list.append(scorer.score(pred_otkd_lines[i].strip(),label_otkd_lines[i].strip())['rouge1'].fmeasure)

100%|██████████| 2997/2997 [00:06<00:00, 491.63it/s]


In [79]:
print('OTKD:',np.mean(otkd_rouge_scorer_rouge1_list))
print('NCLS:',np.mean(ncls_rouge_scorer_rouge1_list))
print('MCLAS:',np.mean(mclas_rouge_scorer_rouge1_list))

OTKD: 0.37716121880163717
NCLS: 0.3866214169465592
MCLAS: 0.35344512244020043


In [104]:
quirky_indices=[]
for i in trange(len(otkd_rouge_scorer_rouge1_list)):
    if otkd_rouge_scorer_rouge1_list[i]<=0.6 and otkd_rouge_scorer_rouge1_list[i]>ncls_rouge_scorer_rouge1_list[i]+0.3 and otkd_rouge_scorer_rouge1_list[i]>mclas_rouge_scorer_rouge1_list[i]+0.3:
        quirky_indices.append(i)

100%|██████████| 2997/2997 [00:00<00:00, 1036386.27it/s]


In [106]:
quirky_indices

[166, 647, 1601, 1711, 1942, 2125, 2381, 2550, 2832]

In [116]:
for idx in quirky_indices:    
    print('NCLS:',pred_ncls_lines[idx])
    print()
    print('MCLAS:',pred_mclas_lines[idx])
    print()
    print('OTKD:',pred_otkd_lines[idx])
    print()
    print('Gold:',label_otkd_lines[idx])
    print()
    print('Mono Gold:',label_mono_lines[idx])
    print('='*125)

NCLS: pupils who did not finish their homework on time were slapped by their teachers

MCLAS: pupils were slapped by their teachers for failing to complete their homework on time

OTKD: hefei primary school students were slapped by teachers for failing to complete their homework

Gold: a hefei primary school teacher compensated 10 , 000 yuan for slapping students

Mono Gold: 合肥 一 小学 老师 打 学生 一 耳光 赔偿 1万 元
NCLS: korean media : chinese fishermen are tired of being kidnapped in the waters of the dprk

MCLAS: why korean fishermen are reluctant to " near boats "

OTKD: chinese fishermen in dandong , liaoning province : fishing boats can ' t survive without " fishing boats "

Gold: dandong fishing " boat fare " under gun muzzle : fishermen can not survive without north korea

Mono Gold: 枪口 下 的 丹东 捕鱼 “ 傍 艇 费 ” ： 渔民 不 傍 朝鲜 无法 生存
NCLS: " dao - snatching youth " can ' t open the moral bonus mouth

MCLAS: " knife - snatching youth " can ' t open the mouth of moral plus points

OTKD: " dao - snatchi

In [65]:
print('Gold:',ncls_gold_lines[idx])

Gold: russian real - human version of " barbie doll "


### Representations

In [117]:
ncls_model_path='/vinai/thongnt66/Repos/cross-lingual/MCLAS/ncls_model_abs_en2zh_noseg/model_step_130000.pt'
mclas_model_path='/vinai/thongnt66/Repos/cross-lingual/MCLAS/mclas_model_abs_en2zh_noseg/model_step_130000.pt'
otkd_model_path='/vinai/thongnt66/Repos/cross-lingual/cross-lingual-lx/snap/continual_NCLS_en2zh-en2zh-ot_loss/model_step_130000.pt'